In [1]:
import redis

In [2]:
db = redis.Redis(host='localhost', port=6379, db=15)

In [3]:
keys = db.keys()

In [4]:
len(keys)

18330

In [5]:
import json

In [6]:
keys[0]

b'pergerakan-pergerakan'

In [8]:
d = json.loads(db[keys[0]])

In [9]:
from tqdm import tqdm

selected = {}

for k in tqdm(keys):
    d = json.loads(db[k])
    if not len(d['definisi']):
        continue
        
    selected[k.decode()] = d

100%|██████████████████████████████████████████████████████████████████████████████████| 18330/18330 [00:00<00:00, 33706.51it/s]


In [10]:
len(selected)

20

In [11]:
import re
from unidecode import unidecode

alphabet = 'qwertyuiopasdfghjklzxcvbnm'
consonants = 'bcdfghjklmnpqrstvwxyz'
vowels = 'aeiou'
PUNCTUATION = '!"#$%&\'()*+,./:;<=>?@[\]^_`{|}~'

In [12]:
import enchant
d = enchant.Dict('en_US')

In [13]:
results = set()

for data in tqdm(selected.values()):
    
    texts = []
    for definisi in data['definisi']:
        texts.extend(definisi.split('Definisi : ')[1].split())

    if data['tesaurus']:
        texts.extend(data['tesaurus'])
    
    for k in range(len(texts)):
        
        t = texts[k]
        t = unidecode(t)
        t = t.replace('--', '-')
        for p in PUNCTUATION:
            t = t.replace(p, '')

        if all([c in consonants for c in t]):
            continue

        if len(t) <= 3:
            continue

        if len(re.findall(r'\d+', t)):
            continue

        if t[0] in '-':
            continue

        if t in ['yg', 'utk']:
            continue
            
        if t[-1] in '-':
            continue
        
        if 'dll' in t:
            continue
            
        t = t.lower()
        
        if d.check(t):
            continue
            
        if t in results:
            continue
            
        results.add(t)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 5713.92it/s]


In [14]:
len(results)

143

In [15]:
results

{'amat',
 'anggota',
 'atau',
 'badan',
 'baik',
 'bangat',
 'batang',
 'berasa',
 'bergalur-galur',
 'bergaris-garis',
 'berkait',
 'berkambi',
 'berkedip-kedip',
 'berkelip-kelip',
 'berlarah-larah',
 'bermongkok',
 'biasanya',
 'buah',
 'bubuh',
 'cabang',
 'colour',
 'colouring',
 'colours',
 'darat',
 'demam',
 'dewan',
 'dibahagikan',
 'digunakan',
 'dikesipnya',
 'edisi',
 'ekzos',
 'elektronik',
 'elok',
 'garam',
 'gula-gula',
 'halus',
 'hasil',
 'hendak',
 'hidup',
 'hingga',
 'ilmu',
 'inggeris-melayu',
 'injap',
 'jalu',
 'jenisnya',
 'kambi',
 'kamus',
 'kapal',
 'kayu',
 'keadaan',
 'kebebasan',
 'kedua',
 'keempat',
 'kelegaan',
 'kelihatantampak',
 'keluar',
 'keluasan',
 'kemudahan',
 'kerana',
 'komponen',
 'kulit',
 'kurang',
 'laut',
 'lawas',
 'lebih',
 'lidahnya',
 'litar',
 'makanan',
 'mata',
 'matanya',
 'melarah',
 'melawas',
 'melawaskan',
 'memandu',
 'memasang',
 'membentuk',
 'memberi',
 'membongkok',
 'membuat',
 'memukul',
 'mencelur',
 'mencucuk',
 'me

In [16]:
with open('dbp-v7.json', 'w') as fopen:
    json.dump(selected, fopen)